In [217]:
import pandas as pd

In [363]:
# Read the data
artistas = pd.read_csv('Artistas.csv', sep=',')
artistas = artistas.drop_duplicates()

entradas = pd.read_csv('Entradas Cortesia.csv', sep=',')
entradas = entradas.drop_duplicates()

eventos = pd.read_csv('Eventos.csv', sep=',')
eventos = eventos.drop_duplicates()

hosp_traslados = pd.read_csv('Hospedajes y Traslados.csv', sep=',')
hosp_traslados = hosp_traslados.drop_duplicates()

productoras = pd.read_csv('Productoras.csv', sep=',')
productoras = productoras.drop_duplicates()

tour = pd.read_csv('Tour.csv', sep=',')
tour = tour.drop_duplicates()


artistas = artistas.rename(columns={'Nombre artistico': 'Artista'})
entradas = entradas.rename(columns={'evento': 'Evento'})
entradas = entradas.rename(columns={'artista': 'Nombre'})

In [364]:
artistas['ID_artista'] = pd.Series(range(1, len(artistas) + 1), index=artistas.index)
artistas['ID_artista'] = artistas['ID_artista'].apply(lambda x: 'A' + str(x))

entradas['ID_entrada'] = pd.Series(range(1, len(entradas) + 1), index=entradas.index)
entradas['ID_entrada'] = entradas['ID_entrada'].apply(lambda x: 'E' + str(x))

eventos['ID_evento'] = pd.Series(range(1, len(eventos) + 1), index=eventos.index)
eventos['ID_evento'] = eventos['ID_evento'].apply(lambda x: 'E' + str(x))

productoras['ID_productora'] = pd.Series(range(1, len(productoras) + 1), index=productoras.index)
productoras['id_productora'] = productoras['ID_productora'].apply(lambda x: 'P' + str(x))

tour['ID_tour'] = pd.Series(range(1, len(tour) + 1), index=tour.index)
tour['ID_tour'] = tour['ID_tour'].apply(lambda x: 'T' + str(x))

In [365]:
artistas = artistas.rename(columns={'Artista': 'Nombre', 'id': 'ID_artista', 'Inicio carrera': 'Fecha_inicio', 'Nº contacto': 'Contacto'})
artistas['Artista'] = artistas['Nombre'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
artistas = artistas.drop(['Artista'], axis=1)
entradas = entradas.merge(artistas[['Nombre', 'ID_artista']], on='Nombre', how='left')
entradas = entradas.drop(['Nombre'], axis=1)

In [367]:
entradas.to_csv('entradas_bdd.csv', index=False)
artistas.to_csv('artistas_bdd.csv', index=False)

In [370]:
entradas

,Evento,asiento,ID_entrada,ID_artista
0,Bangerz,D15,E1,A157
1,I Aint Worried,A17,E2,A173
2,Show Forest,A12,E3,A145
3,Bang Bang,C30,E4,A126
4,DJ Snake,D12,E5,A46
...,...,...,...,...
140,Say you wont let go,A30,E141,A108
141,Epic Show,A29,E142,A104
142,Que oso,B28,E143,A169
143,Fruta y te,C0,E144,A28
